In [1]:
import warnings
warnings.filterwarnings('ignore')
import yaml
import pandas as pd

In [2]:
from multi_agents import MultiAgentSystem

In [3]:
with open("/Users/sigmoid/Downloads/llm_keys.yaml", "r") as f:
    llm_keys = yaml.safe_load(f)

In [4]:
file_path = '/Users/sigmoid/Documents/PepsiCo - Gen AI PoC/lift_bot/src'

In [28]:
# Load data (MODIFIED for new structure)
df_expenses = pd.read_csv(
    f"{file_path}/data/Expenses_RB.csv",
    usecols=[
        'Region',
        'Country',
        'Category',
        'Brand',
        'Year',
        'Time Month',
        'Tier 1',
        'Tier 2',
        'Tier 3',
        'Expense Status', 
        'Pending At', 
        'Expense Logged by (NS)', 
        'Audit Status',
        'Audit Comments',
        'Pep Share (USD)', 
        'Bottler Share (USD)',
        'Total Expense (USD)'
    ]
)
# Preprocessing
for col in ['Pep Share (USD)', 'Bottler Share (USD)', 'Total Expense (USD)']:
    df_expenses[col] = df_expenses[col].fillna(0)
    df_expenses[col] = df_expenses[col].astype(int)
for col in ['Expense Status', 'Audit Status']:
    df_expenses[col] = df_expenses[col].str.title()
# Renaming columns
df_expenses = df_expenses.rename(
    columns={
        'Time Month': 'Month', 'Expense Logged by (NS)': 'Expense Logged by',
        'Pep Share (USD)': 'Pep Expense', 
        'Bottler Share (USD)': 'Bottler Expense', 
        'Total Expense (USD)': 'Total Expense'
    }
)

In [29]:
df_expenses.head()

,Expense Status,Pending At,Expense Logged by,Audit Status,Audit Comments,Tier 1,Tier 2,Tier 3,Year,Month,Country,Region,Category,Brand,Pep Expense,Bottler Expense,Total Expense
0,Approved,NaN,Bottler,NaN,NaN,STB - Push,Unilateral,Capital Equipment,2024,10,ARGENTINA,LAB South,Energy Drinks,Rockstar,417,397,814
1,Approved,NaN,Bottler,NaN,NaN,STB - Push,Unilateral,CDAs,2024,5,ARGENTINA,LAB South,Sport Drinks,Gatorade,159,151,310
2,Approved,NaN,Bottler,NaN,NaN,STB - Push,Unilateral,CDAs,2024,2,ARGENTINA,LAB South,CSD,Pepsi,104665,99679,204345
3,Approved,NaN,Bottler,NaN,NaN,STB - Push,Unilateral,CDAs,2024,9,ARGENTINA,LAB South,CSD,Pepsi,9532,9078,18610
4,Approved,NaN,Bottler,NaN,NaN,STB - Push,Unilateral,Capital Equipment,2024,10,ARGENTINA,LAB South,Sport Drinks,Gatorade,3507,3340,6848


In [30]:
filtered_df = df_expenses[
    (df_expenses['Brand'].str.strip().str.upper() == 'PEPSI') &
    (df_expenses['Country'].str.strip().str.upper() == 'MEXICO') &
    (df_expenses['Year'] == 2025) &
    (df_expenses['Month'] >= 1) & (df_expenses['Month'] <= 7)
]

In [31]:
filtered_df['Total Expense'].sum()

np.int64(3283126)

In [32]:
filtered_df.groupby(['Tier 1'], as_index=False)['Total Expense'].sum()

,Tier 1,Total Expense
0,Pull-Non-Working,2049
1,Pull-Working,1469304
2,STB - Push,1811773


In [22]:
filtered_df['Year'].unique()

array([2025, 2024])

In [7]:
df_budget = pd.read_csv(
    f"{file_path}/data/Budget_RB.csv",
    usecols=[
        "Region",
        "Country", 
        "Year", 
        "Category", 
        "Brand", 
        "Tier 1",
        "Tier 2", 
        "Tier 3", 
        "Pep Budget", 
        "Bottler Budget", 
        "Budget"
    ]
)
# Preprocessing
for col in ['Pep Budget', 'Bottler Budget', 'Budget']:
    df_budget[col] = df_budget[col].fillna(0)
    df_budget[col] = df_budget[col].astype(int)
# Renaming columns
df_budget = df_budget.rename(
    columns={
        'Budget': 'Total Budget'
    }
)

In [8]:
df_budget.head()

,Region,Country,Year,Brand,Tier 1,Tier 2,Tier 3,Total Budget,Pep Budget,Bottler Budget,Category
0,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Ad Production,Digital Ad Production,0,0,0,Water
1,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Ad Production,Other Ad Production,0,0,0,Water
2,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Ad Production,TV Print Radio OOH Prod,491755,491755,0,Water
3,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Agency Fees,Creative Agency Fees,0,0,0,Water
4,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Agency Fees,In Store and POS Design/Development,0,0,0,Water


In [9]:
multi_agent_obj = MultiAgentSystem(
    model_name='gpt-4o',
    api_key=llm_keys['open_ai'],
    expense_dataset=df_expenses,
    budget_dataset=df_budget,
    plot_path='jupyter_plots',
)

In [24]:
# Category must also be included

In [10]:
# Expense
# question = 'What are the Tier 1 expenses in 2024 for Brazil?'
# question = "How are the Tier 1 expenses in CY varying for Mexico & Brazil for Pepsi brand?"

In [11]:
# Budget
question = 'What is the total budget allocated for Mexico in 2025?'
# question = "Can you give me split of Tier 1 items for each Brazil & Mexico for 2025?"

In [12]:
result = multi_agent_obj.graph.nodes['supervisor'].invoke(
    {
        'question': question
    }
)

In [13]:
result

{'result': {'thought_process': 'The user is asking for specific budget data for Mexico in 2025. This requires data analysis to retrieve the budget information for that year and region. The Insight Agent is best suited for this task as it can analyze budget data and provide the required information.',
  'next': 'Insight Agent',
  'enriched_question': 'What is the total budget allocated for Mexico in 2025?'},
 'messages': [AIMessage(content='The user is asking for specific budget data for Mexico in 2025. This requires data analysis to retrieve the budget information for that year and region. The Insight Agent is best suited for this task as it can analyze budget data and provide the required information.', additional_kwargs={}, response_metadata={}, name='supervisor')],
 'next': 'Insight Agent'}

In [14]:
result2 = multi_agent_obj.graph.nodes['Insight Agent'].invoke(
    {
        'enriched_question': result['result']['enriched_question']
    }
)



> Entering new AgentExecutor chain...

Invoking: `analyze_budget_data` with `Calculate the total budget allocated for Mexico in 2025.`


Response Text:
<approach>
To calculate the total budget allocated for Mexico in 2025, I will:
1. Filter the dataset for entries where the "Country" is "Mexico" and the "Year" is 2025.
2. Sum the "Total Budget" column for these filtered entries to get the total budget allocated for Mexico in 2025.
</approach>

<code>
import pandas as pd

# Normalize the 'Country' and 'Year' columns for comparison
df['Country'] = df['Country'].str.strip().str.upper()
df['Year'] = df['Year'].astype(int)

# Filter the dataset for Mexico in 2025
filtered_df = df[(df['Country'] == 'MEXICO') & (df['Year'] == 2025)]

# Calculate the total budget
total_budget_mexico_2025 = filtered_df['Total Budget'].sum()
total_budget_mexico_2025
</code>

<answer>
The total budget allocated for Mexico in 2025 is {total_budget_mexico_2025}.
</answer>
-----------------------------------------

In [15]:
result2.keys()

dict_keys(['input', 'history', 'output', 'recorder_steps', 'agent'])

In [16]:
result2['output']

'<answer>The total budget allocated for Mexico in 2025 is $5,742,565,224.</answer><graph>None</graph>'

In [17]:
len(result2['recorder_steps'])

2

In [18]:
len(result2['recorder_steps'][-2:])

2

In [19]:
result2['input']

'What is the total budget allocated for Mexico in 2025?'

In [20]:
result2['recorder_steps'][0]['observation']['approach']

'To answer this question, I will need to:\n1. Filter the dataset for entries where the "Country" is "Mexico" and the "Year" is 2025.\n2. Sum the "Total Budget" for these filtered entries to get the total budget allocated for Mexico in 2025.'

In [21]:
result2['recorder_steps'][0]['observation']['answer']

'The total budget allocated for Mexico in 2025 is 5742565224.'

In [22]:
result2['recorder_steps'][1]

{'final_answer': '<answer>The total budget allocated for Mexico in 2025 is $5,742,565,224.</answer><graph>None</graph>'}

In [23]:
result2['output'] == result2['recorder_steps'][1]['final_answer']

True

In [24]:
question

'What is the total budget allocated for Mexico in 2025?'

In [25]:
result3 = multi_agent_obj.graph.invoke(
    {
        'question': question
    }
)



> Entering new AgentExecutor chain...

Invoking: `analyze_budget_data` with `Calculate the total budget allocated for Mexico in 2025.`


Response Text:
<approach>
To calculate the total budget allocated for Mexico in 2025, I will:
1. Filter the dataset for entries where the "Country" is "Mexico" and the "Year" is 2025.
2. Sum the "Total Budget" column for these filtered entries to get the total budget allocated for Mexico in 2025.
</approach>

<code>
import pandas as pd

# Assuming df is the dataframe containing the budget data
# Normalize the 'Country' and 'Year' columns for comparison
df['Country'] = df['Country'].str.strip().str.upper()
df['Year'] = df['Year'].astype(int)

# Filter the dataframe for Mexico in 2025
filtered_df = df[(df['Country'] == 'MEXICO') & (df['Year'] == 2025)]

# Calculate the total budget
total_budget_mexico_2025 = filtered_df['Total Budget'].sum()
total_budget_mexico_2025
</code>

<answer>
The total budget allocated for Mexico in 2025 is {total_budget_mexic

In [26]:
result3

{'result': {'thought_process': "The Insight Agent has provided the total budget allocated for Mexico in 2025. The user's query has been answered with the specific data requested.",
  'next': 'FINISH',
  'direct_response': 'The total budget allocated for Mexico in 2025 is $5,742,565,224.',
  'enriched_question': 'What is the total budget allocated for Mexico in 2025?'},
 'messages': [AIMessage(content="The Insight Agent has provided the total budget allocated for Mexico in 2025. The user's query has been answered with the specific data requested.", additional_kwargs={}, response_metadata={}, name='supervisor')],
 'next': 'FINISH'}